In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance

In [81]:
def calculate_overall_credit_score(credit_scores, weights = None):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    
    max_score = 100
    min_score = 0
    
    if weights:
        overall_credit_score = sum(score * weight for score, weight in zip(credit_scores, weights)) / sum(weights)
    
    else:
        overall_credit_score = sum(credit_scores) / len(credit_scores)

    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if overall_credit_score >= 75:
        overall_credit_rating = "Good"
    elif overall_credit_score >= 50:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return overall_credit_score, overall_credit_rating

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the actual credit score within a universal score range based on where the financial ratio
    falls within its thresholds, alongside the credit rating, confidence level, and universal range scores.
    
    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the actual credit score (int), credit rating (str),
               confidence level (float), and universal score ranges (dict).
    """
    # Adjust thresholds and financial ratio for inverse relationships
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio
        industry_thresholds = {k: 1/v for k, v in industry_thresholds.items()}
        expert_thresholds = {k: 1/v for k, v in expert_thresholds.items()}
    
    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2])
        for rating in ['good', 'fair', 'poor']
    }
    
    credit_rating = min(distances, key=distances.get)
    
    # Universal score ranges for each rating category
    score_ranges = {
        'good': (75, 100),
        'fair': (50, 74),
        'poor': (0, 49)
    }

    # Determine the min and max threshold for the financial ratio
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())
    
    # Calculate where the financial ratio falls within its thresholds
    financial_ratio = max(min_threshold, min(financial_ratio, max_threshold))
    ratio_position = (financial_ratio - min_threshold) / (max_threshold - min_threshold)
    
    # Apply this proportion to the corresponding universal score range
    credit_score_range = score_ranges[credit_rating]
    score_range_width = credit_score_range[1] - credit_score_range[0]
    credit_score = credit_score_range[0] + ratio_position * score_range_width
    
    # Calculate confidence level
    confidence_level = (credit_score - credit_score_range[0]) / score_range_width
    confidence_level = max(0, min(1, confidence_level))
    
    return credit_score, credit_rating, confidence_level, score_ranges

In [82]:
company_ratios = {
    'A': [0.2, 0.4, 1.5, 0.6, 1.2],
    'B': [0.9, 1.1, 1.3, 0.7, 0.8],
    'C': [1.0, 0.6, 0.1, 0.9, 0.2]
}

# Define industry and expert thresholds for each financial ratio
industry_thresholds = [
    {'good': 1.0, 'fair': 0.7, 'poor': 0.4},
    {'good': 0.6, 'fair': 0.4, 'poor': 0.2},
    {'good': 1.2, 'fair': 0.9, 'poor': 0.6},
    {'good': 0.8, 'fair': 0.6, 'poor': 0.4},
    {'good': 1.1, 'fair': 0.8, 'poor': 0.5}
]

expert_thresholds = [
    {'good': 1.2, 'fair': 0.8, 'poor': 0.5},
    {'good': 0.7, 'fair': 0.5, 'poor': 0.3},
    {'good': 1.4, 'fair': 1.0, 'poor': 0.7},
    {'good': 0.9, 'fair': 0.7, 'poor': 0.5},
    {'good': 1.3, 'fair': 0.9, 'poor': 0.6}
]

# Define inverse relationship for each financial ratio
inverse_relationships = [False, True, False, True, False]

In [83]:
df = pd.read_excel("metrics.xlsx", index_col=0)

In [84]:
df

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
BLU SJ Equity,32.809328,6.346575,4.215240,1.150158,7.547418
MTN SJ Equity,65.116785,8.547827,7.405181,1.305329,13.114130
TKG SJ Equity,56.593172,3.495262,6.401370,1.430977,8.796523
VOD SJ Equity,65.004940,9.790341,16.143841,0.793112,13.355185


In [85]:
df.describe()

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,54.881056,7.045001,8.541408,1.169894,10.703314
std,15.246373,2.761873,5.240340,0.276201,2.968725
min,32.809328,3.495262,4.215240,0.793112,7.547418
25%,50.647211,5.633747,5.854837,1.060896,8.484247
50%,60.799056,7.447201,6.903275,1.227743,10.955327
75%,65.032901,8.858455,9.589846,1.336741,13.174394
max,65.116785,9.790341,16.143841,1.430977,13.355185


In [96]:
industry_thresholds = [
    {'good': 50.647211, 'fair': 60.799056, 'poor': 65.032901},
    {'good': 8.858455, 'fair': 7.447201, 'poor': 5.633747},
    {'good': 9.589846, 'fair': 6.903275, 'poor': 5.854837},
    {'good': 1.060896, 'fair': 1.227743, 'poor': 1.336741},
    {'good': 13.174394, 'fair': 10.955327, 'poor': 8.484247}
]

expert_thresholds = [
    {'good': 50.647211, 'fair': 60.799056, 'poor': 65.032901},
    {'good': 8.858455, 'fair': 7.447201, 'poor': 5.633747},
    {'good': 9.589846, 'fair': 6.903275, 'poor': 5.854837},
    {'good': 1.060896, 'fair': 1.227743, 'poor': 1.336741},
    {'good': 13.174394, 'fair': 10.955327, 'poor': 8.484247}
]

company_ratios = {
    "BLU SJ Equity": [32.80932765,  6.34657542,  4.21523991,  1.15015783,  7.54741826],
    "MTN SJ Equity": [65.11678489,  8.54782697,  7.4051809 ,  1.30532892, 13.11412971],
    "TKG SJ Equity": [56.59317159,  3.49526179,  6.40136975,  1.43097657,  8.79652344],
    "VOD SJ Equity": [65.00493974,  9.79034064, 16.14384136,  0.79311212, 13.35518524],
    
}

inverse_relationships = [True, False, False, True, False]

In [109]:
# Iterate over each company and calculate credit scores for each financial ratio
for company, ratios in company_ratios.items():
    print(f"Company {company}:")
    credit_scores = []

    for i, ratio in enumerate(ratios):
        credit_score, credit_rating, confidence_level, range_scores = calculate_credit_score(
            ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
        )

        credit_scores.append(credit_score)

#         print(f"  Financial Ratio {i+1}:")
        print(f"    Credit Score: {credit_score}")
        #print(f"    Credit Rating: {credit_rating}")
        # print(f"    Confidence Level: {confidence_level:.2%}")
        # print(f"    Range Scores:")
        # for rating, scores in range_scores.items():
        #     print(f"      {rating.capitalize()}: {scores[0]:.2f} - {scores[1]:.2f}")

    overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores)
    print(f"Overall Credit Score: {overall_credit_score:.2f}")
    print(f"Overall Credit Rating: {overall_credit_rating}")
    print()
    

Company BLU SJ Equity:
    Credit Score: 100.0
    Credit Score: 10.831552059907443
    Credit Score: 0.0
    Credit Score: 64.97386675975177
    Credit Score: 0.0
Overall Credit Score: 35.16
Overall Credit Rating: Poor

Company MTN SJ Equity:
    Credit Score: 0.0
    Credit Score: 97.59181273157135
    Credit Score: 59.96202515174663
    Credit Score: 4.535033614397527
    Credit Score: 99.67877184872884
Overall Credit Score: 52.35
Overall Credit Rating: Fair

Company TKG SJ Equity:
    Credit Score: 62.60087109256218
    Credit Score: 0.0
    Credit Score: 53.511848565826746
    Credit Score: 0.0
    Credit Score: 3.2624874145735707
Overall Credit Score: 23.88
Overall Credit Rating: Poor

Company VOD SJ Equity:
    Credit Score: 0.07420467980688664
    Credit Score: 100.0
    Credit Score: 100.0
    Credit Score: 100.0
    Credit Score: 100.0
Overall Credit Score: 80.01
Overall Credit Rating: Good



In [92]:
df.T

,BLU SJ Equity,MTN SJ Equity,TKG SJ Equity,VOD SJ Equity
tot_debt_to_tot_eqy,32.809328,65.116785,56.593172,65.004940
interest_coverage_ratio,6.346575,8.547827,3.495262,9.790341
return_on_asset,4.215240,7.405181,6.401370,16.143841
tot_debt_to_ebitda,1.150158,1.305329,1.430977,0.793112
ebitda_to_tot_int_exp,7.547418,13.114130,8.796523,13.355185


In [102]:
inverse_relationships

[True, False, False, True, False]

In [112]:
df.quantile(.75)

tot_debt_to_tot_eqy        65.032901
interest_coverage_ratio     8.858455
return_on_asset             9.589846
tot_debt_to_ebitda          1.336741
ebitda_to_tot_int_exp      13.174394
Name: 0.75, dtype: float64

In [114]:
ranges = np.array([50.647211, 60.799056, 65.032901])
act_ranges = np.array([65.004940, 65.004940, 65.004940])

In [116]:
abs(ranges - act_ranges)

array([14.357729,  4.205884,  0.027961])

In [117]:
(2 + 2) / 2

2.0

In [118]:
ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]

(13.35518524,
 {'good': 13.174394, 'fair': 10.955327, 'poor': 8.484247},
 {'good': 13.174394, 'fair': 10.955327, 'poor': 8.484247},
 False)